In [1]:
!pip list | grep lang

langchain                0.0.263
langchainplus-sdk        0.0.17
langsmith                0.0.22


In [2]:
!pip list | grep openai

openai                   0.27.8


### LangChain Expression Language
In this notebook we will have a try on the latest feature released by langchain, using pipeline to link model, prompt and any function or parser you want.


#### 1. Toy example
We will first take a look at a simplest example to link a LLM and a prompt. Before we take a look at Langchain expression language, let's take a look at pipe package which support pipe operation in python

In [3]:
from pipe import select, where

numbers = [1, 2, 3, 4, 5]
# find even numbers and multiple by 3
results = list(numbers |  where(lambda x: x % 2 == 0) | select(lambda x: x * 3))

results

[6, 12]

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv

# load env for llm service
load_dotenv()

True

In [5]:
# create llm
llm = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

# create prompt template
prompt = ChatPromptTemplate.from_template("Show me description for company {company_name} in 100 words.")

# chain model and prompt
chain = prompt | llm

chain

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[], last=AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api

In [6]:
chain.invoke({"company_name": "HSBC"})

AIMessage(content="HSBC Holdings plc is a British multinational investment bank and financial services holding company. It was founded in 1865 and has since grown to become one of the world's largest banking and financial services organizations with operations in over 60 countries. HSBC provides a range of financial products and services such as personal and business banking, wealth management, corporate and investment banking, and global markets. The company is committed to sustainability and aims to help its customers and communities to thrive, while also managing its operations in a responsible and sustainable way. HSBC is headquartered in London, UK.", additional_kwargs={}, example=False)

In [7]:
# a chain with a string output parser
chain_with_output_parser = prompt | llm | StrOutputParser()
chain_with_output_parser

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api_version

In [8]:
chain_with_output_parser.invoke({"company_name": "HSBC"})

"HSBC is a multinational banking and financial services company headquartered in London, United Kingdom. With a history dating back to 1865, HSBC is one of the world's largest banks, operating in over 65 countries with a network of over 3,900 branches. The company provides a range of financial services to individuals, businesses, and institutions, including commercial banking, retail banking, wealth management, and investment banking. HSBC is committed to sustainable finance and is a leader in environmental, social, and governance (ESG) investing. The company also supports a range of community initiatives and charitable organizations around the world."

#### 2. Attaching Function Call information

We are not call the function acctually here, we are just using the function specification and let LLM to figure out the input parameter for the function

In [32]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# functions = [
#     {
#       "name": "save_description",
#       "description": "save the description for the a company",
#       "parameters": {
#         "type": "object",
#         "properties": {
#           "company_name": {
#             "type": "string",
#             "description": "The name of the company"
#           },
#           "description": {
#             "type": "string",
#             "description": "The description of the company"
#           }
#         },
#         "required": ["company_name", "description"]
#       }
#     }
#   ]


# chain = (
#     prompt 
#     | llm.bind(function_call= {"name": "save_description"}, functions= functions) 
#     | JsonOutputFunctionsParser()
# )

# chain.invoke({"company_name": "HSBC"})

The output should be look like, bue since our OPENAI model at Azure is still the old GPT 3.5 which does not support functional call and it will encounter an error.

{ 
  'company_name' : 'HSBC',
  'description' : 'HSBC Holdings plc is a multinational banking and financial services company headquartered in London, United Kingdom. It is one of the largest banking organizations in the world, with a presence in 64 countries and territories, serving more than 40 million customers. HSBC offers a wide range of......'
}

### 3. Runnable in Langchain

LangChain Expression Language (LEL) provides a convenient and straightforward approach for writing components in a declarative manner and effortlessly combining them. To gain a better understanding of its functionality, let's examine its runnable interface.

LangChain introduces a runnable interface that serves as a blueprint for implementing various components. Subclasses of this interface are required to implement the following functions:
- stream: handles streaming output
- invoke: processes single input
- batch: handles batch input
- astream: handles asynchronous streaming
- ainvoke: performs asynchronous invocation
- abatch: handles asynchronous batch processing

In [9]:
from langchain.schema.runnable import Runnable

# the ChatPromptTemplate implemented Runnable as well
print(isinstance(prompt, Runnable))

True


In [10]:
# let's write our own component

from langchain.load.serializable import Serializable
from langchain.schema.runnable import RunnableConfig, RunnablePassthrough
from langchain.schema.runnable.base import Input, Runnable, Optional


class StdOutputRunnable(Serializable, Runnable[Input, Input]):

    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Input, config: Optional[RunnableConfig] = None) -> Input:
        print(f"I recevied input ->  {input} ")
        return self._call_with_config(lambda x: x, input, config)


runnable_chain =  RunnablePassthrough() | StdOutputRunnable()

runnable_chain.invoke({'company_name':'HSBC'})

I recevied input ->  {'company_name': 'HSBC'} 


{'company_name': 'HSBC'}

In [11]:
chain = prompt | StdOutputRunnable() | llm

chain.invoke({'company_name': 'Alibaba'})

I recevied input ->  messages=[HumanMessage(content='Show me description for company Alibaba in 100 words.', additional_kwargs={}, example=False)] 


AIMessage(content="Alibaba Group Holding Limited is a Chinese multinational technology company that specializes in e-commerce, retail, internet, and technology. Founded in 1999 by Jack Ma, Alibaba has grown to become one of the largest companies in the world, with operations in over 200 countries and regions. The company's platforms include Taobao, Tmall, AliExpress, and Alibaba.com, which offer a range of services, including online marketplaces, payment systems, cloud computing, and digital media. Alibaba's mission is to make it easy for businesses to do business anywhere, and it has played a significant role in driving the growth of e-commerce in China and beyond.", additional_kwargs={}, example=False)

#### 4. Retriever

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough

# create embedding model
embeddings = OpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    model="text-embedding-ada-002"
)

# Create the retriever
vectorstore = Chroma.from_texts(["harrison worked at kensho"], embedding=embeddings)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [14]:
chain.invoke("where did harrison work?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Harrison worked at Kensho.'